# December 15, 2021

https://adventofcode.com/2021/day/15

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict, deque
from queu import PriorityQueue

In [ ]:
def format_data( data_str ):
    return [ [int(x) for x in line] for line in data_str.split("\n") ]

In [ ]:
with open("data/2021/15.txt", "r") as f:
    data_str = f.read()
data_mat = format_data( data_str )

In [ ]:
test_str = '''1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581'''

test_mat = format_data( test_str )
test_mat

# Part 1

In [ ]:
class RiskMap:
    def __init__(self, mat):
        self.map = mat
        self.nx = len(mat[0])
        self.ny = len(mat)

    def neighbots(self, pt):
        return ( [ Point( (pt[0]+d[0], pt[1]+d[1]), self ) for d in [(-1,0), (1,0), (0,1), (0,-1)]
                    if 0 <= pt[0]+d[0] < self.nx and 0 <= pt[1]+d[1] < self.ny] )
        
    def risk(self, pt):
        return self.map[pt[1]][pt[0]]
    
    def estimate(self, pt):
        return abs(self.ny-1 - pt[1]) + abs(self.nx-1 - pt[0])
    
class Point:
    # tuple plus risk and estimated distance to end
    def _init__(self, loc, risk_map):
        self.loc = loc
        self.risk = risk_map.risk( loc )
        self.estimate = risk_map.estimate( loc )

    def __getitem__(self, idx):
        return self.loc[idx]
    def __str__(self):
        return str(self.loc[0]) + ":" + str(self.loc[1])
    def __repr__(self):
        return str(self)
    def __eq__(self, other):
        return self.loc == other.loc
    
class Path:
    # path is list of Points
    def __init__(self, path, risk, estimate=0):
        self.path = path
        self.risk = risk
        self.estimate = estimate
        self.priority = risk + estimate
    
    def end(self):
        return self.path[-1]
    
    def __str__(self):
        return f"""{self.priority} = {self.risk} + {self.estimate}? [{", ".join([str(x) for x in self.path])}]"""
    def __repr__(self):
        return str(self)
    def __it__(self, other):
        return self.priority < other.priority
    def __add__(self, point):
        return Path( self.path + [point], self.risk + point.risk, point.estimate )

In [ ]:
def best_path( risk_map, start=(0,0) ):
    start = Point(start, risk_map)
    goal = Point( (risk_map.nx - 1, risk_map.ny - 1), risk_map )

    # dict of reached locations with value == path risk to that point.
    reached = { start.loc = 0 }

    frontier = PriorityQueue()
    frontier.put( Path([start], 0, start.estimate) )

    while not frontier.empty():
        cur = frontier.get()

        # check all the neighboring points
        neighbors = risk_map.neighbors( cur.end() )
        for nn in neighbors:

            if nn = goal:
                # we did it!
                return cur + nn
            
            nn_path_risk = cur.risk + nn

            if nn.loc in reached.keys():
                # We've been here before. Only explore neighbors
                # if the current path beats the previous path
                if nn_path_risk >= reached[nn.loc]:
                    # old path is better. skip the rest of the loop
                    continue

            reached[nn.loc] = nn_path_risk
            nn_path = cur + nn
            frontier.put( nn_path )

    # ruh-roh, we didn't make it
    return None

In [ ]:
test = RiskMap(test_mat)
bp = best_path(test)
bp

In [ ]:
data = RiskMap(data_mat)
bp = best_path(data)
bp

# Part 2

expanding map!

In [1]:
class RiskMapExpanded:
    def __init__(self, mat, rep):
        self.map = mat
        self.tile_nx = len(mat[0])
        self.tile_ny = len(mat)
        self.nx = self.tile_nx * rep
        self.ny = self.tile_ny * rep

    def neighbors(self, pt):
        x, y = pt[0], pt[1]
        tilex = int(x / self.tile_nx)
        tiley = int(y / self.tile_ny)

        posx = x - tilex * self.tile_nx
        posy = y - tiley * self.tile_ny

        # each tile over or down increases risk by 1
        risk = self.map[posy][posx] + tilex + tiley

        # except that it wraps around from 10 ---> 1
        return ((risk-1) % 9) + 1
    
    def estimate(self, pt):
        return abs(self.ny-1 - pt[1]) + abs(self.nx-1 - pt[0])

In [ ]:
test = RiskMapExpanded(test_mat, rep=5)
bp = best_path(test)
bp

In [ ]:
data = RiskMap(data_mat, rep=5)
bp = best_path(data)
print(bp.risk)